### Load Dataset

In [1]:
import torch as T
import os
import numpy as np

In [2]:
pos_dir = './dataset/train/positive/'
neg_dir = './dataset/train/negative/'

In [3]:
images = []

In [4]:
for pos_img in os.listdir(pos_dir):
    images.append(pos_dir + pos_img)

In [5]:
for neg_img in os.listdir(neg_dir):
    images.append(neg_dir + neg_img)

In [6]:
labels = len(os.listdir(pos_dir)) * [1] + len(os.listdir(neg_dir)) * [0]

### Calculate Means & Stds

In [7]:
from PIL import Image

In [8]:
img_size = 224

In [9]:
means = []
stds = []

for image in images:
    
    image = Image.open(image)
    image = image.convert('RGB')
    image = image.resize((img_size, img_size))
    image = np.array(image) / 255
    image = image.transpose((2, 0, 1)).reshape((3, img_size*img_size))
    
    means.append(np.mean(image, axis=1))
    stds.append(np.std(image, axis=1))

In [10]:
mean = np.mean(np.array(means), axis=0)
std = np.mean(np.array(stds), axis=0)

In [11]:
print(mean, std)

[0.80289211 0.78376652 0.77243479] [0.23013637 0.24434427 0.25245189]


### Train-Validation Split

In [12]:
import math

In [13]:
np.random.seed(0)

In [14]:
dataset_len = len(labels)

indices = np.arange(dataset_len)
np.random.shuffle(indices)

train_idx = indices[:math.floor(0.9*dataset_len)]
val_idx = indices[math.floor(0.9*dataset_len):]

In [15]:
val_images = [images[i] for i in val_idx]
val_labels = [labels[i] for i in val_idx]

In [16]:
train_images = [images[i] for i in train_idx]
train_labels = [labels[i] for i in train_idx]

### Create PyTorch Loaders

In [17]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt

In [18]:
class DigikalaDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Normalize((0.80289211, 0.78376652, 0.77243479), (0.23013637, 0.24434427, 0.25245189)),
        ])
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        
        image = Image.open(self.images[index])
        image = image.convert('RGB')
        image = image.resize((img_size, img_size))
        image = np.array(image) / 255
        image = image.transpose((2, 0, 1))
        image = T.tensor(image, dtype=T.float32)
        image = self.transform(image)
        
        label = self.labels[index]
        label = T.tensor(label, dtype=T.float32)
        
        return image, label

In [19]:
def get_loader(images, labels, batch_size=32, shuffle=True):
    dataset = DigikalaDataset(images, labels)
    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

In [20]:
train_loader = get_loader(train_images, train_labels, batch_size=25)
val_loader = get_loader(val_images, val_labels, batch_size=5)

### Define The Model

In [21]:
import torch.nn as nn

In [22]:
model = T.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)

Using cache found in /home/hossein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/hossein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/hossein/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


In [23]:
model.classifier.fc = nn.Linear(1280, 1)

### Train The Model

In [24]:
import torch.optim as optim
import time

In [25]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [26]:
device = 'cuda'
criterion = criterion.to(device)
model = model.to(device)

In [27]:
def binary_accuracy(preds, y):
    rounded_preds = T.round(T.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [28]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
        
    for i, (imgs, lbs) in enumerate(iterator):
        
        if i % 100 == 0:
            print(f'    mini-batch {i}')
        
        optimizer.zero_grad()
        
        imgs = imgs.to(device)
        lbs = lbs.to(device)
        
        predictions = model(imgs).squeeze()
        loss = criterion(predictions, lbs)
        acc = binary_accuracy(predictions, lbs)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item() * len(lbs)
        epoch_acc += acc.item() * len(lbs)
                
    return epoch_loss / len(iterator.dataset), epoch_acc / len(iterator.dataset)

In [29]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
        
    for i, (imgs, lbs) in enumerate(iterator):
        
        imgs = imgs.to(device)
        lbs = lbs.to(device)
        
        predictions = model(imgs).squeeze()
        loss = criterion(predictions, lbs)
        acc = binary_accuracy(predictions, lbs)
        
        epoch_loss += loss.item() * len(lbs)
        epoch_acc += acc.item() * len(lbs)
                
    return epoch_loss / len(iterator.dataset), epoch_acc / len(iterator.dataset)

In [30]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [31]:
N_EPOCHS = 4

for epoch in range(N_EPOCHS):

    print(f'Epoch: {epoch+1}')
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
       
    print(f'Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'    Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'     Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print()

Epoch: 1
    mini-batch 0
    mini-batch 100
    mini-batch 200
    mini-batch 300
Epoch Time: 2m 56s
    Train Loss: 0.336 | Train Acc: 86.59%
     Val. Loss: 0.308 |  Val. Acc: 89.64%

Epoch: 2
    mini-batch 0
    mini-batch 100
    mini-batch 200
    mini-batch 300
Epoch Time: 3m 3s
    Train Loss: 0.263 | Train Acc: 90.28%
     Val. Loss: 0.649 |  Val. Acc: 78.58%

Epoch: 3
    mini-batch 0
    mini-batch 100
    mini-batch 200
    mini-batch 300
Epoch Time: 3m 5s
    Train Loss: 0.228 | Train Acc: 91.49%
     Val. Loss: 0.290 |  Val. Acc: 89.99%

Epoch: 4
    mini-batch 0
    mini-batch 100
    mini-batch 200
    mini-batch 300
Epoch Time: 2m 58s
    Train Loss: 0.197 | Train Acc: 93.20%
     Val. Loss: 0.314 |  Val. Acc: 91.50%



In [29]:
del model
T.cuda.empty_cache()

### Saving The Model

In [69]:
T.save(model.state_dict(), 'eff_91.torch')

### Loading The Model

In [ ]:
model = T.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=False)
model.classifier.fc = nn.Linear(1280, 1)
model.load_state_dict(T.load('eff_91.torch'))

### Loading Test Data

In [70]:
test_dir = './dataset/test/'

In [71]:
test_images = []
for test_img in os.listdir(test_dir):
    test_images.append(test_dir + test_img)

In [72]:
test_labels = len(os.listdir(test_dir)) * [0]

In [73]:
test_loader = get_loader(test_images, test_labels, batch_size=5, shuffle=False)

### Predicting

In [74]:
def predict(model, iterator):
    
    model.eval()
    
    preds = []
    
    with T.no_grad():
        for i, (imgs, lbs) in enumerate(iterator):

            imgs = imgs.to(device)        
            predictions = model(imgs).squeeze()
            binary_predictions = T.round(T.sigmoid(predictions))
            preds += binary_predictions.tolist()
        
    return preds

In [75]:
preds = predict(model, test_loader)

In [76]:
preds = [int(p) for p in preds]

### Saving to File

In [77]:
import pandas as pd

In [78]:
pred_df = pd.DataFrame()

In [79]:
pred_df['name'] = [t[len('./dataset/test/'):] for t in test_images]

In [80]:
pred_df['predicted'] = preds

In [81]:
pred_df.head(5)

,name,predicted
0,2c039159-56fa-4798-b0a2-adeddd61c327.jpg,1
1,178c77d7-47f4-4790-850a-c873c8f05597.jpg,0
2,8da4b8e2-a41a-4a10-b2aa-9f9a4481e74a.jpg,1
3,038369e8-c33f-43b6-b71f-16a6fd75ec83.jpg,0
4,443ea086-4d3c-4e94-9107-22cc6d1a5b44.jpg,0


In [82]:
pred_df.to_csv('output.csv', index=False)